## Concepts of MapReduce in Dask

### What is MapReduce?

MapReduce is a programming model and an associated implementation for processing and generating big data sets with a parallel, distributed algorithm on a cluster. [Wikipedia](https://en.wikipedia.org/wiki/MapReduce)



A MapReduce program is composed of a map procedure (or method), which performs filtering, sorting or element elaborations (such as sorting out students by first name, one queue for each name), and a reduce method, which performs a summary operation (such as counting the number of students in each queue, yielding name frequencies). The MapReduce  frameowork it's essential when you have big data problems and a cluster.
It's realiable and fault tolerant and allows to work on huge data (Exabyte) on a cluster without memory issues.

The model is a specialization version of the ```split-apply-combine``` approache for data analysis. Each action of the map or reduce function is made on a partition of data with an indipendet process.
Each worker of a cluster execute the same operations on separated chunks of data. Once the computation is terminated the results are retrieved from each worker node and then they are combined toghter. If necessary the map-reduce function are applied again on the results combination. This process is repeated until all the operations are done and all the results are merge into single one which is the real final results.


 ![title](img/The-MapReduce-architecture-MapReduce-Algorithm-There-are-four-steps-to-implement.png)

Let's see an example of the Map function. Try to convert the ```CRSDepTime``` column to a timestamp. In the dataset the CRSDepTime which is a timestamps as HHMM, it is stored as integers in the csv:

In [3]:
from distributed import Client
client = Client() #Client('dask-scheduler:8786')

In [4]:

import os
import dask
import dask.dataframe as dd

df = dd.read_csv(os.path.join('data', 'nycflights', '*.csv'),
                 parse_dates={'Date': [0, 1, 2]},
                 dtype={'TailNum': str,
                        'CRSElapsedTime': float,
                        'Cancelled': bool})



crs_dep_time = df.CRSDepTime.head(100)
crs_dep_time

0     1055
1     1055
2     1055
3     1729
4     1729
      ... 
95    1855
96    1855
97    1855
98    1855
99    1855
Name: CRSDepTime, Length: 100, dtype: int64

In order to convert these to timestamps of scheduled departure time, we need to convert these integers into ```pd.Timedelta``` objects, and then combine them with the ```Date``` column.

In [5]:
%time
import pandas as pd

# Get the first 10 dates to complement our `crs_dep_time`
date = df.Date.head(100)

# Get hours as an integer, convert to a timedelta
hours = crs_dep_time // 100
hours_timedelta = pd.to_timedelta(hours, unit='h')

# Get minutes as an integer, convert to a timedelta
minutes = crs_dep_time % 100
minutes_timedelta = pd.to_timedelta(minutes, unit='m')

# Apply the timedeltas to offset the dates by the departure time
departure_timestamp = date + hours_timedelta + minutes_timedelta
departure_timestamp

Wall time: 0 ns


0    1993-01-29 10:55:00
1    1993-01-30 10:55:00
2    1993-01-31 10:55:00
3    1993-01-03 17:29:00
4    1993-01-04 17:29:00
             ...        
95   1993-01-22 18:55:00
96   1993-01-24 18:55:00
97   1993-01-25 18:55:00
98   1993-01-26 18:55:00
99   1993-01-27 18:55:00
Length: 100, dtype: datetime64[ns]

Let's try with the MapReduce.
Dask.dataframe provides a few methods to make applying custom functions to Dask DataFrames easier:

+ map_partitions
+ map_overlap
+ reduction

Here we'll just be discussing map_partitions, which we can use to implement to_timedelta on our own:

In [6]:
%time 
hours = df.CRSDepTime // 100
# hours_timedelta = pd.to_timedelta(hours, unit='h')
hours_timedelta = hours.map_partitions(pd.to_timedelta, unit='h')

minutes = df.CRSDepTime % 100
# minutes_timedelta = pd.to_timedelta(minutes, unit='m')
minutes_timedelta = minutes.map_partitions(pd.to_timedelta, unit='m')

departure_timestamp = df.Date + hours_timedelta + minutes_timedelta

Wall time: 0 ns


In [7]:
departure_timestamp

Dask Series Structure:
npartitions=4
    datetime64[ns]
               ...
               ...
               ...
               ...
dtype: datetime64[ns]
Dask Name: add, 36 tasks

In [8]:
departure_timestamp.head()

0   1993-01-29 10:55:00
1   1993-01-30 10:55:00
2   1993-01-31 10:55:00
3   1993-01-03 17:29:00
4   1993-01-04 17:29:00
dtype: datetime64[ns]

### Exercise1: 
Try to rewrite the above code to use a single call to map_partitions.

#### Hints:
All code of the function will be executed on chunks/partition of data

In [14]:
def compute_departure_timestamp(df):
    hours = df.CRSDepTime // 100
    hours_timedelta = pd.to_timedelta(hours, unit='h')

    minutes = df.CRSDepTime % 100
    minutes_timedelta = pd.to_timedelta(minutes, unit='m')

    departure_timestamp = df.Date + hours_timedelta + minutes_timedelta

    return df


In [15]:
departure_timestamp = df.map_partitions(compute_departure_timestamp)


In [16]:
departure_timestamp.head()

,Date,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,TailNum,ActualElapsedTime,...,AirTime,ArrDelay,DepDelay,Origin,Dest,Distance,TaxiIn,TaxiOut,Cancelled,Diverted
0,1993-01-29,5,1055.0,1055,1228.0,1212,US,66,NaN,93.0,...,NaN,16.0,0.0,EWR,BUF,282.0,NaN,NaN,False,0
1,1993-01-30,6,1052.0,1055,1214.0,1212,US,66,NaN,82.0,...,NaN,2.0,-3.0,EWR,BUF,282.0,NaN,NaN,False,0
2,1993-01-31,7,1103.0,1055,1213.0,1212,US,66,NaN,70.0,...,NaN,1.0,8.0,EWR,BUF,282.0,NaN,NaN,False,0
3,1993-01-03,7,1736.0,1729,1838.0,1831,US,70,NaN,62.0,...,NaN,7.0,7.0,LGA,SYR,198.0,NaN,NaN,False,0
4,1993-01-04,1,1730.0,1729,1825.0,1831,US,70,NaN,55.0,...,NaN,-6.0,1.0,LGA,SYR,198.0,NaN,NaN,False,0


In [17]:
client.close()